In [1]:
from pathlib import Path
from nestmodel.load_datasets import load_fg_dataset_cached


from nestmodel.fast_graph import FastGraph
import numpy as np
from datetime import datetime
import pickle
import matplotlib.pyplot as plt

In [2]:
datasets = ["karate", 
            "phonecalls",
            "HepPh", 
            "AstroPh", 
#            "web-Google",
#             "soc-Pokec"
#            "deezer_HR", "deezer_HU", "deezer_RO","tw_musae_DE",
#            "tw_musae_ENGB","tw_musae_FR","lastfm_asia","fb_ath",
#            "fb_pol", "facebook_sc"
           ]

In [3]:
dataset_path = None # set path here

In [4]:
from nestmodel.utils import AutoList
from tqdm.auto import tqdm
import time

def run_rewires(Rewirer, phis, n_rewires, n_samples, n_test_flips=10):
    output = AutoList({"SAE", "J", "ratio", "result_p", "SAE", "rew_time"})
    Rewirer.validate_params(phis)
    Rewirer.reset_graph()
    # do rewiring now to discard potential compile overhead
    result = Rewirer.rewire(1, phis[0], 0)
    
    t0 = time.process_time()
    result = Rewirer.rewire(0, phis[0], 0)
    setup_time = time.process_time()-t0
    
    t0 = time.process_time()
    result = Rewirer.rewire(n_test_flips, phis[0], 0)
    flip_time = time.process_time()-t0
    
    #if delta * n_rewires > 10:
    one_sample_time = max(setup_time + (flip_time - setup_time) * n_rewires/n_test_flips, 0)
    one_sample_unit = "[s]"
    if one_sample_time>3600:
        one_sample_time /= 3600
        one_sample_unit = "[h]"
    print("approx time for 1 sample:", one_sample_time, one_sample_unit)
    print("approx total time:", one_sample_time * len(phis) * n_samples, one_sample_unit)
    for phi in phis:
        for seed in tqdm(range(n_samples), miniters=1, leave=False):
            Rewirer.reset_graph()

            result = Rewirer.rewire(n_rewires, phi, seed)

            Rewirer.log_result(result, output)

            time.sleep(0.001) # required to ensure the status bar is updated properly
    return output

In [5]:
def save_data(output, prefix):
    if n_rewires == 1:
        print("skipping saving for n_samples = 1")
        return
    time_prefix = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print("saving", dataset, prefix,  time_prefix)
    
    with open(f"./results/{prefix}_{dataset}_{time_prefix}.pkl", "wb") as f:
        pickle.dump((dataset, n_samples, n_rewires, output.SAE, output.J, output.ratio, output.rew_time), f)

## ERGM Sampling

In [6]:
from nestmodel.ERGM_experiments import ERGM_RewireWrapper, NeSt_RewireWrapper, Erdos_RewireWrapper

In [7]:
dataset = "karate"
verbosity=1
phis = np.array([1, 10, 100])
n_samples = 100
if dataset == "karate":
    n_edges = 78
    n_rewires = 200
    phis *= 34
elif dataset == "phonecalls":
    n_edges = 45_680
    n_rewires = 100_000
    phis *= 36_595
else:
    raise ValueError()


In [ ]:
"""ERGM """

G = load_fg_dataset_cached(dataset_path, dataset)


if dataset != "karate":
    Rewirer = ERGM_RewireWrapper(G, "dict")
else:  
    Rewirer = ERGM_RewireWrapper(G.to_nx(), "adjacency")

    
output = run_rewires(Rewirer, phis, n_rewires, n_samples)
save_data(output, "ERGM")

# phonecalls sampling time 4184.89127677201s

approx time for 1 sample: 1.1578430609969452 [h]
approx total time: 347.35291829908357 [h]


## NeSt sampling

In [ ]:
if dataset == "karate":
    depths = [0, 1, 2]
elif dataset == "phonecalls":
    depths = [0, 1, 2, 3, 4, 5, 6, 7, 8]
else:
    raise ValueError()

In [ ]:
G = load_fg_dataset_cached(dataset_path, dataset)
G.check_results = False
Rewirer = NeSt_RewireWrapper(G.copy())
G.ensure_edges_prepared()


output2 = run_rewires(Rewirer, depths, n_rewires, n_samples)
save_data(output2, "NeSt")

## Erdos Renyi

In [ ]:
G = load_fg_dataset_cached(dataset_path, dataset)
G.check_results = False
Rewirer = Erdos_RewireWrapper(G.copy())
G.ensure_edges_prepared()


output2 = run_rewires(Rewirer, [0], n_rewires, n_samples)
save_data(output2, "Erdos")